Decrypt passwords.json?

In [ ]:
var path = require('path');
var fs = require('fs');
var crypto = require('crypto');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var PASS_FILE = path.join(PROFILE_PATH, '.credentials', 'password.txt');
var PASSWORDS_FILE = path.join(PROFILE_PATH, '.credentials', 'passwords.json');

function decrypt(text) {
    var pass = process.env.SELENIUM_PASS || fs.readFileSync(PASS_FILE).toString();
    var decipher = crypto.createDecipher('aes-256-ctr', pass);
    var dec = decipher.update(text, 'hex', 'binary');
    dec += decipher.final('binary');
    return dec;
}

function getCredentials(name) {
    var resultSet = {};
    var passwords = JSON.parse(fs.readFileSync(PASSWORDS_FILE).toString());
    var set = passwords.filter(el => el.host == name)[0] || {};
    for (var i in set) {
        if (set.hasOwnProperty(i)) {
            if (i == 'added' || i == 'host') {
                resultSet[i] = set[i];
                continue;
            }
            resultSet[i] = decrypt(set[i]);
        }
    }
    return resultSet;
};
module.exports = getCredentials;
getCredentials;



Add encrypted to passwords.json?



In [ ]:
var fs = require('fs');
var crypto = require('crypto');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var PASS_FILE = path.join(PROFILE_PATH, '.credentials', 'password.txt');
var PASSWORDS_FILE = path.join(PROFILE_PATH, '.credentials', 'passwords.json');
var pass = process.env.SELENIUM_PASS || fs.readFileSync(PASS_FILE).toString();

function encrypt(text) {
    var cipher = crypto.createCipher('aes-256-ctr', pass);
    var crypted = cipher.update(text, 'binary', 'hex');
    crypted += cipher.final('hex');
    return crypted;
}

function encryptSet(set) {
    var resultSet = {};
    for (var i in set) {
        if (set.hasOwnProperty(i)) {
            if (i == 'added' || i == 'host') {
                resultSet[i] = set[i];
                continue;
            }
            resultSet[i] = encrypt(set[i]);
        }
    }
    return resultSet;
}

function saveCredentials(passwordAddJson) {
    // TODO: passAdd from file?
    var passwords = JSON.parse(fs.readFileSync(PASSWORDS_FILE).toString());
    passwordAddJson.added = new Date();
    var encrypted = encryptSet(passwordAddJson);
    if (passwordAddJson.host) {
        passwords = passwords.filter(function (el) {
            return el.host.toLowerCase() != passwordAddJson.host.toLowerCase()
                && (el.username || '').toLowerCase() != passwordAddJson.username.toLowerCase();
        });
    }
    passwords[passwords.length] = encrypted;
    var passwordString = JSON.stringify(passwords, null, 4);
    //var compressed = zlib.gzipSync(passwordString).toString('base64');
    console.log('Passwords: Saving encrypted file');
    fs.writeFileSync('passwords-new.json', passwordString);
    fs.renameSync(PASSWORDS_FILE, 'passwords-backup.json');
    fs.renameSync('passwords-new.json', PASSWORDS_FILE);
    fs.unlinkSync('passwords-backup.json');
    if (typeof passAdd !== 'undefined' && fs.existsSync(passAdd)) {
        console.log('Passwords: Deleting input file: ' + passAdd);
        fs.unlinkSync(passAdd);
    }
    return encrypted;
}

module.exports = saveCredentials;
saveCredentials;


In [5]:
var importer = require('../Core');
var saveCredentials = importer.import('add encrypted passwords.json');

saveCredentials({
    host: 'linkedin.com',
    password: '********',
    username: 'megamindbrian@gmail.com'
});


reading notebook /Users/briancullinan/jupytangular2/Selenium/service auth.ipynb
compiling /Users/briancullinan/jupytangular2/Selenium/service auth.ipynb[1]
Passwords: Saving encrypted file


{ host: 'linkedin.com',
  password: '746668b6ce2e94495c7b',
  username: '5d623eb3c73ec81e1f313c41e9845d978cd242b859a157',
  added: 2018-06-09T01:21:27.116Z }

(node:25220) Warning: Use Cipheriv for counter mode of aes-256-ctr
(node:25220) Warning: Use Cipheriv for counter mode of aes-256-ctr


In [ ]:
// add all passwords from passwords.html?
var importer = require('../Core');
var fs = require('fs');
var saveCredentials = importer.import('add encrypted passwords.json');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = PROFILE_PATH + '/Documents/passwords.htm';

function readPasswordsHtm() {
    var credentials;
    var passwords = fs.readFileSync(project).toString('utf16le');
    importer.regexToArray(/TBODY[^>]*>[\s\S]*?\/TBODY/ig, passwords).forEach(b => {
        var creds = {};
        creds['host'] = (((/subcaption[^>]*>(.*?)<\/td>/ig)
            .exec(b) || [])[1] || '').replace(/<wbr>/ig, '').toLowerCase();
        if (creds.host.trim() === '') {
            return;
        }
        importer.regexToArray(/<tr>[\s\S]*?<\/tr>/ig, b).forEach(f => {
            var key = (((/field[^>]*>(.*?)<\/td>/ig)
                .exec(f) || [])[1] || '').replace(/<wbr>/ig, '');
            var value = (((/wordbreakfield[^>]*>(.*?)<\/td>/ig)
                .exec(f) || [])[1] || '').replace(/<wbr>/ig, '');
            if (key.trim() !== '') {
                creds[key] = value;
            }
        });
        credentials = saveCredentials(creds);
    });
    return credentials;
};
module.exports = readPasswordsHtm;

// display login form and add to passwords.json?



download passwords from google?

https://passwords.google.com



In [ ]:
var importer = require('../Core');
var loginGoogle = importer.import('log in google', {client, getCredentials});

function waitForPasswordLoad(r) {
    return Promise.all([
        client.getText(r + ' [role="rowheader"]'),
        client.getText(r + ' [role="rowheader"] ~ [role="gridcell"]:nth-child(2)'),
        client.getValue(r + ' [role="rowheader"] ~ [role="gridcell"]:last-child input')
    ])
        .then(r => r[2] === 'Loading...'
            ? client.pause(1000).then(() => waitForPasswordLoad(r))
            : Promise.resolve(r))
};

function copyPasswordRow(i) {
    const r = '[role="row"]:nth-of-type(' + i + ') ';
    return client.click(r + ' [role="button"][aria-label*="Toggle"]')
        .pause(1000)
        .then(() => waitForPasswordLoad(r))
        .then(r => saveCredentials({
            host: r[0],
            username: r[1],
            password: r[2]
        }))
        .catch(e => console.log(e))
};

function copyPasswords() {
    return client.url('https://passwords.google.com')
        .then(() => loginGoogle(client)
        .pause(1000)
        .elements('[role="row"]')
        .then(els => importer.runAllPromises(els.value.map((e, i) => resolve => {
            return client.isExisting('[role="row"]:nth-of-type(' + i + ') [role="button"][aria-label*="Toggle"]')
                .then(is => is ? copyPasswordRow(i) : client)
                .then(row => resolve(row))
        })));
};

var saveCredentials = importer.import('add encrypted passwords.json');
function downloadGooglePasswords() {
    return copyPasswords();
};
module.exports = downloadGooglePasswords;


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
runSeleniumCell('download passwords from google')
    .then(downloadGooglePasswords => downloadGooglePasswords())
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



How to log in to Google using webdriver?



In [ ]:
var matchUsername = 'input[name="Email"], input[autocomplete="username"]';
function enterGoogleUsername(email) {
    console.log('Google: Sign in required');
    return client
        .pause(1000)
        .then(() => client.$(matchUsername))
        .then(el => el.addValue(email))
        .then(() => client.$('#identifierNext'))
        .then(el => el.click())
        .catch((e) => console.log(e));
}

var matchPassword = 'input[name="Passwd"], input[autocomplete="password"], input[name="password"]';
function enterGooglePassword(pass) {
    console.log('Google: Require password');
    return client
        .pause(2000)
        .then(() => client.$(matchPassword))
        .then(el => el.addValue(pass))
        .then(() => client.$('#passwordNext'))
        .then(el => el.click());
}

function enterCredentials(is) {
    var credentials = getCredentials('accounts.google.com');
    if (is) {
        return client.$('#passwordNext')
            .then(el => el.isDisplayed())
            .then(is => is
                ? enterGooglePassword(credentials.Passwd)
                : enterGoogleUsername(credentials.Email)
                    .then(() => enterGooglePassword(credentials.Passwd)))
    } else {
        return client
            .findElements('xpath', '//h1[contains(., "Choose an account")]')
            .then(is => {
                if (is.length > 0) {
                    return client
                        .findElements('xpath', '//*[@data-authuser][contains(., "@")]')
                        .then(is => {
                            console.log(is);
                            return is.length > 0
                              ? client.$('//*[@data-authuser][contains(., "@")]').then(el => el.click())
                              : []
                        })
                        .then(() => client.$('#passwordNext'))
                        .then(el => el.isDisplayed())
                        .then(is => is
                            ? enterGooglePassword(credentials.Passwd)
                            : [])
                } else {
                    return client.findElements('xpath', '//h1[contains(., "Sign in")]')
                        .then(is => is
                            ? enterGoogleUsername(credentials.Email)
                                .then(() => enterGooglePassword(credentials.Passwd))
                            : null)
                }
            })
            .catch((e) => console.log(e));
    }
}

function loginGoogle() {
    return client
        .findElements('css selector', 'a[href*="ServiceLogin"]')
        .then(is => is.length > 0 ? client.click('a[href*="ServiceLogin"]') : null)
        .then(() => client.pause(1000))
        .then(() => client.findElements('css selector', 'form[action*="signin"]'))
        .then(is => enterCredentials(is.length > 0))
        .catch((e) => {
            console.log(e);
            console.log('Google: Could not log in');
        });
}

module.exports = loginGoogle;



Automatically fill any type of login form using various algorithms?

Try to log in to every site in stored passwords

